In [ ]:
import requests
import re
import pandas as pd
from io import StringIO
import bs4
import icalendar
from datetime import datetime, timedelta
import contextlib
import warnings
import urllib3

# Scrape data from libraries

## Köln (Cologne)

In [ ]:
users = {"Cologne account 1": {"Nr":"aXXX", "PIN": "XXXX"}}

In [ ]:
df_koeln = pd.DataFrame()
for selectedUser in users:
    print(selectedUser)
    s = requests.Session() 
    r1 = s.get("https://katalog.stbib-koeln.de/alswww2.dll/APS_ZONES?fn=MyZone&Style=Portal3&SubStyle=&Lang=GER&ResponseEncoding=utf-8")
    cookie = r1.cookies.get_dict()
    pattern = '<META NAME="ZonesObjName" +CONTENT="(.*)">'
    token = re.findall(pattern, r1.text)
    objname = token[0]
    payload = {'Method': "CheckID", 'SHA1': "", 'ZonesLogin': '1', 'Interlock': objname, "BrowseAsHloc": "", "Style": "Portal3", "SubStyle": "", "Lang": "GER", "ResponseEncoding":"utf-8", "BRWR": users[selectedUser]["Nr"], "PIN":users[selectedUser]["PIN"]}
    r2 = s.post("https://katalog.stbib-koeln.de/alswww2.dll/" + objname, cookies=cookie, data=payload)
    token = re.findall(pattern, r2.text)
    objname = token[0]
    cookie = r2.cookies.get_dict()
    cntMedia = int(re.findall('<td class="AccountSummaryCounterValueCellStripe">(.*?)</td>', r2.text)[0])
    i = 0
    cntRenewed = 0
    while cntMedia > 0:
        cntMedia -= 10
        r3 = s.get(f"https://katalog.stbib-koeln.de/alswww2.dll/{objname}?Style=Portal3&SubStyle=&Lang=GER&ResponseEncoding=utf-8" + ("?Method=ShowLoans" if i == 0 else "&Method=PageDown&PageSize=10") , cookies=cookie)
        token = re.findall(pattern, r3.text)
        objname = token[0]
        cookie = r3.cookies.get_dict()
        tbl = re.findall('<table id="BrowseList".*?>.*<\/table>.*SearchResultsTrailer', r3.text, re.MULTILINE | re.DOTALL)
        soup = bs4.BeautifulSoup(tbl[0], 'lxml')
        results = soup.find(attrs = {'id': 'BrowseList'})    
        for row in results.find_all('tr', recursive=False):
            df_row = []
            for col in row.find_all('td', recursive=False):
                table = col.find_all('table')
                if table:
                    df_row = pd.read_html(StringIO(str(col)))[0].transpose()
                    df_row.columns = df_row.iloc[0]
                    df_row = df_row.iloc[1:,:]
                    df_row["Konto"] = selectedUser
                    df_row["Fälligkeitsdatum"] = pd.to_datetime(df_row["Fälligkeits datum"].str.split(" ").apply(lambda x: x[0]), format="%d/%m/%Y")
                    df_row["Item"] = df_row.apply(lambda x: re.findall(r"testLoanSelected\('(.*)',", x["Fälligkeits datum"]), axis=1)
                    df_row["Item"] = df_row["Item"].apply(lambda x: x[0] if x else None)
                    df_row["ButtonTitle"] = [a["title"] for a in col.find_all("a", id=["buttonRenewLoan", "buttonCannotRenewLoan"])]
                    df_row["Verlängerbar"] = df_row.apply(lambda x: "?" not in x["Fälligkeits datum"] or "Medium vorgemerkt" not in x
                    ["ButtonTitle"], axis=1)
                    if "Anz.Verl." not in df_row.columns:
                        df_row["Anz.Verl."] = 0
                    df_row["Anz.Verl."].fillna(0, inplace=True)
                    df_today = df_row[(df_row["Fälligkeitsdatum"].dt.date <= datetime.now().date()) & df_row["Verlängerbar"]]
                    if df_today.size > 0:
                        r4 = s.get(f"https://katalog.stbib-koeln.de/alswww2.dll/{objname}?Style=Portal3&SubStyle=&Lang=GER&ResponseEncoding=utf-8&Method=Renew&Style=Portal3&Item={df_row['Item'].values[0]}", cookies=cookie)
                        if "Die Verlängerung wurde erfolgreich durchgeführt." not in r4.text:
                            print("Verlängerung fehlgeschlagen")
                        else:
                            cntRenewed += 1
                            df_row["Fälligkeitsdatum"] = pd.to_datetime(re.findall("spätestens am <B> (.*) </B> zurück.", r4.text)[0], format="%d/%m/%Y")
                            df_row["Anz.Verl."] += 1
                    if df_koeln.size == 0:
                        df_koeln = df_row
                    else:
                        df_koeln = pd.concat([df_koeln, df_row])
        i += 1

    print(f"{cntRenewed} Medien verlängert")

if df_koeln.size == 0:
    print("Keine Medien gefunden")
else:
    df_koeln["Verlängerbar"] = df_koeln["Verlängerbar"] & (2-df_koeln["Anz.Verl."].astype(int) > 0)
    df_koeln.rename({"Publikationsform": "Medientyp"}, axis=1, inplace=True)
    df_koeln["Max.Verl."] = 2
    df_koeln["Anz.Verl."].fillna(0, inplace=True)
    df_koeln.dropna(axis=1, inplace=True)
    df_koeln["Ausleih- Datum"] = pd.to_datetime(df_koeln["Ausleih- Datum"], format="%d/%m/%Y")
    df_koeln["Bibliothek"] = "Köln"
    # Calculate maximum possible date from due date, extension count and media type
    df_koeln["Max. Datum"] = df_koeln.apply(lambda x: x["Fälligkeitsdatum"] + timedelta((x["Max.Verl."]-int(x["Anz.Verl."])) * 7 * (2 if re.match("DVD|Games|Video-Spiel", x["Medientyp"]) else 4)), axis=1)
df_koeln

## Kreuztal

In [ ]:
users = {"Kreuztal account 1": {"Nr":"XXX", "PIN": "XX.XX.XXXX"}}

In [ ]:
df_kreuztal = pd.DataFrame()
for selectedUser in users:
    print(selectedUser)
    df_rows = pd.DataFrame()
    s = requests.Session() 
    payload = {"koha_login_context":"opac", "userid": users[selectedUser]["Nr"], "password":users[selectedUser]["PIN"]}
    tries = 0
    while tries < 2:
        tries += 1
        r2 = s.post("https://sb-kreuztal.lmscloud.net/cgi-bin/koha/opac-user.pl", data=payload)
        tbl = re.findall('<table id="checkoutst".*<\/table>', r2.text, re.MULTILINE | re.DOTALL)
        if len(tbl) == 0:
            break
        df_rows = pd.read_html(tbl[0], extract_links="body")[0]
        df_rows["Konto"] = selectedUser

        df_rows["Fälligkeitsdatum"] = pd.to_datetime(df_rows["Fällig"].str[0].str.replace("Fälligkeitsdatum:  ", ""), format="%d.%m.%Y")
        df_today = df_rows[(df_rows["Fälligkeitsdatum"].dt.date <= datetime.now().date()) & df_rows["Verlängern"].str[1].notnull()]
        for idx,row in df_today.iterrows():
            r3 = s.get("https://sb-kreuztal.lmscloud.net/" + row["Verlängern"][1])
            if r3.status_code != 200:
                print("Verlängerung fehlgeschlagen")
        if df_today.size == 0:
            break
        else:
            print("Verlängert: " + str(df_today.size) + " Medien")     

    if df_kreuztal.size == 0:
        df_kreuztal = df_rows
    else:
        df_kreuztal = pd.concat([df_kreuztal, df_rows])

if df_kreuztal.size == 0:
    print("Keine Medien gefunden")
else:
    df_kreuztal["Max.Verl."] = 2
    df_kreuztal["Anz.Verl."] = 2-df_kreuztal["Verlängern"].str[0].str.extract(".*(\d) von.*").fillna(0).astype(int)
    df_kreuztal["Verfasser"].fillna("Kein Autor", inplace=True)
    df_kreuztal["Medientyp"] = df_kreuztal["Medientyp"].str[0].replace("Medientyp:  ", "")
    df_kreuztal["Verlängerbar"] = ~df_kreuztal["Verlängern"].str[0].str.contains("Nicht verlängerbar") & (2-df_kreuztal["Anz.Verl."] > 0)
    df_kreuztal["Bibliothek"] = "Kreuztal"
    # Calculate maximum possible date from due date, extension count and media type
    df_kreuztal["Max. Datum"] = df_kreuztal.apply(lambda x: x["Fälligkeitsdatum"] + timedelta((x["Max.Verl."]-int(x["Anz.Verl."])) * 7 * (2 if re.match("DVD|Games|Video-Spiel", x["Medientyp"]) else 4)), axis=1)
    df_kreuztal["Titel"] = df_kreuztal["Titel"].str[0]
    df_kreuztal["Verfasser"] = df_kreuztal["Verfasser"].str[0]
df_kreuztal

# UB Hagen

In [ ]:
users = {"Hagen account 1": {"Nr":"aXXX", "PIN": "XXXX"}}

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import locale
locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
df_hagen = pd.DataFrame()

for selectedUser in users:
    print(selectedUser)
    options = Options()
    options.add_argument("--headless")     
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get("https://fub-hagen.digibib.net/intro/account")
    driver.find_element(By.ID, "username").send_keys(users[selectedUser]["Nr"])
    driver.find_element(By.ID, "password").send_keys(users[selectedUser]["PIN"])
    driver.find_element(By.NAME, "_eventId_proceed").click()
    driver.find_element(By.ID, "navddlabel1").click()

    tries = 0
    while tries < 2:
        tries += 1
        driver.get("https://fub-hagen.digibib.net/intro/account")  

        soup = bs4.BeautifulSoup(driver.page_source, 'lxml')
        results = soup.find_all(attrs = {'class': 'list-unstyled d7-result-itemlist'})
        results = pd.read_html(str(results))
        loanids = [a["data-loanid"] for a in soup.find_all(attrs = {'class': 'd7-result-item d7-result-item--intro'})]
        csrftoken = soup.find(attrs = {'name': 'csrf_token'})["value"]
        cookie = driver.get_cookies()
        cookie = {c["name"]: c["value"] for c in cookie}

        df_rows = pd.DataFrame()
        for result in results:
            df_row = result.transpose().iloc[1:,:]            
            df_row.rename(columns=results[0][0], inplace=True)
            df_row.columns = df_row.columns.str.replace(":", "") 
            try:
                df_row["Fälligkeitsdatum"] = pd.to_datetime(df_row["Zurückzugeben bis"], format="%a., %d. %B %Y")
            except:
                df_row["Fälligkeitsdatum"] = pd.to_datetime(df_row["Zurückzugeben bis"], format="%a., %d. %b. %Y")
         
            if df_rows.size == 0:
                df_rows = df_row  
            else:
                df_rows = pd.concat([df_rows, df_row])
                
        df_rows["Konto"] = selectedUser        
        df_rows["LoanID"] = loanids

        df_today = df_rows[(df_rows["Fälligkeitsdatum"].dt.date <= datetime.now().date())]

        for idx,row in df_today.iterrows():
            r3 = s.post("https://fub-hagen.digibib.net/intro/renew/one", data={"csrf_token": csrftoken, "loanId": row["LoanID"]}, cookies=cookie)
            if r3.status_code != 200:
                print("Verlängerung fehlgeschlagen")
        if df_today.size == 0:
            break
        else:
            print("Verlängert: " + str(df_today.shape[0]) + " Medien")    
                
    if df_hagen.size == 0:
        df_hagen = df_rows
    else:
        df_hagen = pd.concat([df_hagen, df_rows])

if df_hagen.size == 0:
    print("Keine Medien gefunden")
else:
    df_hagen["Bibliothek"] = "Hagen"
    #rename Ausgeliehen to Ausleihdatum
    df_hagen.rename({"Ausgeliehen am": "Ausleihdatum"}, axis=1, inplace=True)
    df_hagen.rename({"Signatur": "Verfasser"}, axis=1, inplace=True)
    df_hagen.rename({"Medium": "Titel"}, axis=1, inplace=True)
    df_hagen["Ausleihdatum"] = df_hagen["Ausleihdatum"].str.replace('\.', '', regex=True)
    df_hagen["Ausleihdatum"] = pd.to_datetime(df_hagen["Ausleihdatum"], format="%a, %d %b %Y")
    df_hagen["Max.Verl."] = 9
    df_hagen["Max. Datum"] = df_hagen.apply(lambda x: x["Ausleihdatum"] + timedelta(days=360) , axis=1)
    df_hagen["Anz.Verl."] = round((df_hagen["Fälligkeitsdatum"] - df_hagen["Ausleihdatum"]) / timedelta(days=42), 0).astype(int)
    df_hagen["Verlängerbar"] = df_hagen["Anz.Verl."] < df_hagen["Max.Verl."] #TODO
    df_hagen["Medientyp"] = "Buch"
df_hagen

# Create calendar

In [ ]:
cols = ["Fälligkeitsdatum", "Bibliothek", "Konto", "Verlängerbar", "Anz.Verl.", "Medientyp", "Verfasser", "Titel", "Max. Datum", "Max.Verl."]
df = pd.DataFrame()
if df_koeln.size > 0:
    df = df_koeln[cols]
if df_kreuztal.size > 0:
    df = pd.concat([df, df_kreuztal[cols]]).reset_index(drop=True)
if df_hagen.size > 0:
    df = pd.concat([df[cols], df_hagen[cols]]).reset_index(drop=True)

In [ ]:
cal = icalendar.Calendar()
entriesPerEvent = 25

def format_row(row):
    konto = row["Konto"] + ":\n" if row.name == 0 or row["Konto"] != rows.loc[row.name-1, "Konto"] else ""
    verlaengerbar = "x " if not row["Verlängerbar"] else "- "
    titel = row["Titel"][:25] + (row["Titel"][25:] and ".")
    verfasser = " - " + row["Verfasser"][:25] + (row["Verfasser"][25:] and ".") if row["Verfasser"] != "Kein Autor" else ""
    medientyp = " (" + row["Medientyp"][:7] + (row["Medientyp"][7:] and ".") + ")"
    verlbis = " (" + str(row["Max.Verl."]-int(row["Anz.Verl."])) + "x verl. bis " + row["Max. Datum"].strftime("%d.%m.%Y") + ")" if row["Verlängerbar"] else " (NICHT verlängerbar)"
    return konto + verlaengerbar + titel + verfasser + medientyp + verlbis

for idx, r in df.groupby("Fälligkeitsdatum"):
    for library, rws in r.groupby("Bibliothek"):
        i = 0
        rws.sort_values(["Konto", "Verlängerbar", "Max. Datum"], inplace=True)
        while True:
            rows = rws[i:i+entriesPerEvent].reset_index(drop=True).reindex()
            event = icalendar.Event()
            event.add("summary", library + " (" + str(rows["Verlängerbar"].sum()) + "/" + str(rows["Verlängerbar"].count()) + " Medien verlängerbar)")
            event.add('dtstart', idx.date())
            event.add('dtstamp', datetime.now())
            alarm = icalendar.Alarm()
            alarm.add("action", "DISPLAY")
            alarm.add("trigger", timedelta(days=-1))
            alarm.add("description", "Medien fällig")
            event.add_component(alarm)
            if not rows["Verlängerbar"].all():
                event.add("priority", "1")
                event.add("categories", ["Fällig"])             
            
            description = "Stand: " + datetime.now().strftime("%d.%m.%Y %H:%M Uhr") + "\n" + "\n".join(rows.apply(format_row, axis=1))
            event.add("description", description)
            cal.add_component(event)
            if len(rows) < entriesPerEvent:
                break
            i += entriesPerEvent
cal.to_ical()